In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import lightgbm
from sklearn.metrics import mean_squared_error
import warnings
import gc


warnings.filterwarnings('ignore')
np.random.seed(4590)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the s

In [2]:
test = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/test.csv')

In [3]:
test.head()

,Unnamed: 0,ID,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name,combination_id,in_train,month,year,quarter,date_block_num
0,0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""","(5, 5037)",True,11,2015,4,34
1,1,1,5,5320,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""","(5, 5320)",False,11,2015,4,34
2,2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""","(5, 5233)",True,11,2015,4,34
3,3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""","(5, 5232)",True,11,2015,4,34
4,4,4,5,5268,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""","(5, 5268)",False,11,2015,4,34


In [4]:
sales_train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/sales_train.csv')

In [6]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofweek,dayofmonth,month,quarter,year,minute,hour,total_sales,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.00,1.0,2,2,1,1,2013,0,0,999.00,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,3,3,1,1,2013,0,0,899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.00,-1.0,5,5,1,1,2013,0,0,-899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.05,1.0,6,6,1,1,2013,0,0,1709.05,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1.0,1,15,1,1,2013,0,0,1099.00,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [26]:
data = sales_train[['date_block_num', 'item_price', 'shop_name', 'item_name', 'item_category_name']]

In [27]:
data = pd.concat([data, test[['date_block_num','shop_name', 'item_name', 'item_category_name']]], join="outer")

In [28]:
data.tail()

,date_block_num,item_category_name,item_name,item_price,shop_name
214195,34,Музыка - CD локального производства,СБ. Союз 55,NaN,"Самара ТЦ ""ПаркХаус"""
214196,34,Подарки - Настольные игры,Настольная игра Нано Кёрлинг,NaN,"Самара ТЦ ""ПаркХаус"""
214197,34,Музыка - CD локального производства,НОВИКОВ АЛЕКСАНДР Новая коллекция,NaN,"Самара ТЦ ""ПаркХаус"""
214198,34,Кино - DVD,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),NaN,"Самара ТЦ ""ПаркХаус"""
214199,34,Кино - Blu-Ray,3 ДНЯ НА УБИЙСТВО (BD),NaN,"Самара ТЦ ""ПаркХаус"""


In [30]:
vectorizer = TfidfVectorizer()
vec_item_cat = vectorizer.fit_transform(data.item_category_name)
vectorizer = TfidfVectorizer()
vec_item = vectorizer.fit_transform(data.item_name)
vectorizer = TfidfVectorizer()
vec_shop = vectorizer.fit_transform(data.shop_name)

  (0, 112)	0.6911666948316297
  (0, 98)	0.2111331331436198
  (0, 11)	0.6911666948316297
  (1, 56)	0.3773855157254777
  (1, 96)	0.6305957548948569
  (1, 13)	0.6781807771009148
  (2, 56)	0.3773855157254777
  (2, 96)	0.6305957548948569
  (2, 13)	0.6781807771009148
  (3, 56)	0.3773855157254777
  (3, 96)	0.6305957548948569
  (3, 13)	0.6781807771009148
  (4, 56)	0.3773855157254777
  (4, 96)	0.6305957548948569
  (4, 13)	0.6781807771009148
  (5, 56)	0.3773855157254777
  (5, 96)	0.6305957548948569
  (5, 13)	0.6781807771009148
  (6, 56)	0.3773855157254777
  (6, 96)	0.6305957548948569
  (6, 13)	0.6781807771009148
  (7, 56)	0.3773855157254777
  (7, 96)	0.6305957548948569
  (7, 13)	0.6781807771009148
  (8, 56)	0.3773855157254777
  :	:
  (3150040, 81)	0.6848384150542629
  (3150041, 98)	0.23624508872408345
  (3150041, 71)	0.6893363499190472
  (3150041, 81)	0.6848384150542629
  (3150042, 98)	0.23624508872408345
  (3150042, 71)	0.6893363499190472
  (3150042, 81)	0.6848384150542629
  (3150043, 98)	0.236